Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.599454
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 200: 0.446297
Minibatch accuracy: 81.2%
Validation accuracy: 76.7%
Minibatch loss at step 400: 0.468750
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 600: 0.630007
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%
Minibatch loss at step 800: 0.717297
Minibatch accuracy: 75.0%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 0.459559
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%
Test accuracy: 89.0%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = conv2d(data, layer1_weights)
    pool = max_pool_2x2(conv)
    hidden = tf.nn.relu(pool + layer1_biases)
    conv = conv2d(hidden, layer2_weights)
    pool = max_pool_2x2(conv)
    hidden = tf.nn.relu(pool + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.525250
Minibatch accuracy: 12.5%
Validation accuracy: 10.1%
Minibatch loss at step 200: 0.535867
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 400: 0.566214
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 600: 0.593920
Minibatch accuracy: 75.0%
Validation accuracy: 83.4%
Minibatch loss at step 800: 0.724945
Minibatch accuracy: 81.2%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 0.604585
Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Test accuracy: 89.6%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [10]:
def conv2d(x, W , padding):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding=padding)

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [11]:
# build a convolutional net based on the LeNet 5 architecture.
image_size = 28
num_labels = 10
batch_size = 128
patch_size = 5
depth1 = 6
depth2 = 16
num_hidden1 = 1024
num_hidden2 = 512
starter_learning_rate = 0.05

# Build the complete graph for feeding inputs, training, and saving checkpoints.
graph = tf.Graph()
with graph.as_default():
    # Generate placeholders for the images and labels.
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
    
    # C1: Convolutional layer with 6 feature maps of size 28x28. [28, 28, 6]
    layer1_weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, 1, depth1], stddev=0.1), name='weights')
    layer1_biases = tf.Variable(tf.zeros([depth1]), name='biases')
    #C3: Convolutional layer with 16 feature maps of size 10x10. [10, 10, 16]
    layer3_weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, 6, depth2], stddev=0.1), name='weights')
    layer3_biases = tf.Variable(tf.zeros([depth2]), name='biases')
    # FC5: Fully connected layer with 1024 nodes. [5*5*16, 1024]
    layer5_weights = tf.Variable(tf.truncated_normal(
              [5 * 5 * depth2, num_hidden1], stddev=0.1), name='weights')
    layer5_biases = tf.Variable(tf.zeros([num_hidden1]), name='biases')
    # FC6: Fully connected layer with 512 nodes. [1024 ,512]
    layer6_weights = tf.Variable(tf.truncated_normal(
              [num_hidden1, num_hidden2], stddev=0.1), name='weights')
    layer6_biases = tf.Variable(tf.zeros([num_hidden2]), name='biases')
    #RO7: Read out layer. [512, 10]
    layer7_weights = tf.Variable(tf.truncated_normal(
              [num_hidden2, num_labels], stddev=0.1), name='weights')
    layer7_biases = tf.Variable(tf.zeros([num_labels]), name='biases')
    
    def inference(images):
        
        
        # Layer 1
        # C1: Convolutional layer with 6 feature maps of size 28x28. [28, 28, 6]
        # Each unit of C1 has a 5x5 receptive field in the input layer.
            layer1 = tf.nn.relu(conv2d(images, layer1_weights, 'SAME') + layer1_biases)

        # Layer 2
        # S2: 2X2 Pooling layer with 6 feature maps of size 14x14. [14, 14, 6]
            layer2 = max_pool_2x2(layer1)

        # Layer 3    
        # C3: Convolutional layer with 16 feature maps of size 10x10. [10, 10, 16]
        # Each unit of C3 has a 5x5 receptive field in the input layer.
            layer3 = tf.nn.relu(conv2d(layer2, layer3_weights, 'VALID') + layer3_biases)

        # Layer 4
        # S4: 2X2 Pooling layer with 16 feature maps of size 5x5. [5, 5, 6]
            layer4 = max_pool_2x2(layer3)

        # Layer 5    
        # FC5: Fully connected layer with 1024 nodes. [5*5*16, 1024]
            shape = layer4.get_shape().as_list()
            reshape_l4 = tf.reshape(layer4, [shape[0], shape[1] * shape[2] * shape[3]])    
            layer5 = tf.nn.relu(tf.matmul(reshape_l4, layer5_weights) + layer5_biases)
            layer5_drop = tf.nn.dropout(layer5, 0.8)
        # Layer 6    
        # FC6: Fully connected layer with 512 nodes. [1024 ,512]
            layer6 = tf.nn.relu(tf.matmul(layer5_drop, layer6_weights) + layer6_biases)
            layer6_drop = tf.nn.dropout(layer6, 0.8)
        # Layer 7
        # RO7: Read out layer. [512, 10] 
            logit = tf.matmul(layer6_drop, layer7_weights) + layer7_biases
            return logit
    
    # Build a Graph that computes predictions from the inference model.
    logits = inference(tf_train_dataset)

    ## cacluate loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # use exponential decay for adjusting the learning rate.
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           200, 0.96, staircase=True)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(inference(tf_valid_dataset))
    test_prediction = tf.nn.softmax(inference(tf_test_dataset))

    """
    tf.add_to_collection("images", images_placeholder)  # Remember this Op.
    tf.add_to_collection("labels", labels_placeholder)  # Remember this Op.
    tf.add_to_collection("logits", logits)  # Remember this Op.
    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()
    
    # Uncomment the following line to see what we have constructed.
    # tf.train.write_graph(tf.get_default_graph().as_graph_def(),
    #                      "/tmp", "complete.pbtxt", as_text=True)
    """

In [12]:
#Run training for num_steps and save checkpoint at the end.
num_steps = 10001

with tf.Session(graph=graph) as session:
    # Start the training loop.
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    """
    # Write a checkpoint.
    #checkpoint_file = os.path.join(TRAIN_DIR, 'checkpoint')
    #saver.save(sess, checkpoint_file, global_step=step)
    """

Initialized
Minibatch loss at step 0: 2.821490
Minibatch accuracy: 8.6%
Validation accuracy: 10.1%
Minibatch loss at step 1000: 0.488927
Minibatch accuracy: 85.9%
Validation accuracy: 86.0%
Minibatch loss at step 2000: 0.438351
Minibatch accuracy: 86.7%
Validation accuracy: 87.4%
Minibatch loss at step 3000: 0.473019
Minibatch accuracy: 84.4%
Validation accuracy: 88.5%
Minibatch loss at step 4000: 0.278449
Minibatch accuracy: 91.4%
Validation accuracy: 88.8%
Minibatch loss at step 5000: 0.521940
Minibatch accuracy: 81.2%
Validation accuracy: 89.1%
Minibatch loss at step 6000: 0.272814
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 7000: 0.363099
Minibatch accuracy: 91.4%
Validation accuracy: 89.4%
Minibatch loss at step 8000: 0.257583
Minibatch accuracy: 91.4%
Validation accuracy: 89.6%
Minibatch loss at step 9000: 0.332038
Minibatch accuracy: 89.8%
Validation accuracy: 89.5%
Minibatch loss at step 10000: 0.252641
Minibatch accuracy: 93.0%
Validation accura

In [ ]:
# build a convolutional net based on the LeNet 5 architecture.
image_size = 28
num_labels = 10
batch_size = 16
patch_size = 5
depth = 16
num_hidden1 = 1024
num_hidden2 = 512

def inference(images):
# Layer 1
# C1: Convolutional layer with 6 feature maps of size 28x28. [28, 28, 6]
# Each unit of C1 has a 5x5 receptive field in the input layer.
    with tf.name_scope('layer1'):
        layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, 1, 6], stddev=0.1), name='weights')
        layer1_biases = tf.Variable(tf.zeros([6]), name='biases')
        layer1 = tf.nn.relu(conv2d(images, layer1_weights, 'SAME') + layer1_biases)
    
# Layer 2
# S2: 2X2 Pooling layer with 6 feature maps of size 14x14. [14, 14, 6]
    with tf.name_scope('layer2'):
        layer2 = max_pool_2x2(layer1)
        
# Layer 3    
# C3: Convolutional layer with 16 feature maps of size 10x10. [10, 10, 16]
# Each unit of C3 has a 5x5 receptive field in the input layer.
    with tf.name_scope('layer3'):
        layer3_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, 6, 16], stddev=0.1), name='weights')
        layer3_biases = tf.Variable(tf.zeros([16]), name='biases')
        layer3 = tf.nn.relu(conv2d(layer2, layer3_weights, 'VALID') + layer3_biases)
        
# Layer 4
# S4: 2X2 Pooling layer with 16 feature maps of size 5x5. [5, 5, 6]
    with tf.name_scope('layer4'):
        layer4 = max_pool_2x2(layer3)
        
# Layer 5    
# FC5: Fully connected layer with 1024 nodes. [5*5*16, 1024]
    with tf.name_scope('layer5'):
        shape = layer4.get_shape().as_list()
        reshape_l4 = tf.reshape(layer4, [shape[0], shape[1] * shape[2] * shape[3]])
        layer5_weights = tf.Variable(tf.truncated_normal(
          [shape[1] * shape[2] * shape[3], num_hidden1], stddev=0.1), name='weights')
        layer5_biases = tf.Variable(tf.zeros([num_hidden1]), name='biases')
        layer5 = tf.nn.relu(tf.matmul(reshape_l4, layer5_weights) + layer5_biases)
        
# Layer 6    
# FC6: Fully connected layer with 512 nodes. [1024 ,512]
    with tf.name_scope('layer6'):
        layer6_weights = tf.Variable(tf.truncated_normal(
          [num_hidden1, num_hidden2], stddev=0.1), name='weights')
        layer6_biases = tf.Variable(tf.zeros([num_hidden2]), name='biases')
        layer6 = tf.nn.relu(tf.matmul(layer5, layer6_weights) + layer6_biases)
        
# Layer 7
# RO7: Read out layer. [512, 10] 
    with tf.name_scope('softmax_linear'):
        layer7_weights = tf.Variable(tf.truncated_normal(
          [num_hidden2, num_labels], stddev=0.1), name='weights')
        layer7_biases = tf.Variable(tf.zeros([num_labels]), name='biases')
        logit = tf.matmul(layer6, layer7_weights) + layer7_biases
    return logit